In [1]:
import numpy as np
import pandas as pd
from Utils import *

#crawler
import requests
from bs4 import BeautifulSoup

In [2]:
# 讀入官方的data
df = pd.read_csv('./data/tbrain_train_final_0610.csv')  

In [3]:
df[60:70]

,news_ID,hyperlink,content,name
60,61,https://sina.com.hk/news/article/20190919/1/11...,原標題：龐大董事換血 ### 省略內文 ### 增持計劃不設價格區間。新京報記者王琳琳圖片來...,[]
61,62,https://udn.com/news/story/7238/4250717,台北市長柯文哲(柯P)的母親(柯媽)今天(26)日下午表示 ### 省略內文 ### 一起挺...,[]
62,63,https://m.ctee.com.tw/livenews/lm/ctee/a076720...,2019年區議會選舉24日舉行 ### 省略內文 ### 投票人數亦創回歸以來新高。,[]
63,64,https://news.tvbs.com.tw/politics/1254204,國民黨今天再打民進黨網軍案 ### 省略內文 ### 懷念黨主席吳敦義。（中央社）<BR>更...,[]
64,65,https://www.ettoday.net/amp/amp_news.php?news_...,▲仁愛鄉建設課長盡忠職守 ### 省略內文 ### 公所也會全力協助家屬處理後事。,[]
65,66,https://news.ltn.com.tw/news/society/paper/131...,苗栗縣一群外籍配偶向本報投訴 ### 省略內文 ### 才能認定是否涉及刑事刑責。,['朱小蓉']
66,67,https://news.cnyes.com/news/id/4366465,0台股台股盤勢【華冠投顧】5G族群稍有轉弱持續注意族群連動狀況華冠投顧※來源：華冠投顧201...,[]
67,68,https://m.ctee.com.tw/livenews/ch/chinatimes/2...,動工又停工再推動復工！台中市神岡區浮圳路拓寬工程在前市長胡志強時代動工 ### 省略內文 #...,[]
68,69,https://www.mirrormedia.mg/story/20191223web003/,2019.12.23 03:35中國景氣低迷馬雲：1天接5通借錢電話1週10個朋友賣樓文｜朱...,[]
69,70,https://udn.com/news/story/6812/4126842,10月25日是世界麵食日 ### 省略內文 ### 尤其適合怕胖的人食用。,[]


In [4]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind = np.unique(webKind)
webKind

array(['ccc.technews.tw', 'domestic.judicial.gov.tw', 'ec.ltn.com.tw',
       'ent.ltn.com.tw', 'estate.ltn.com.tw', 'finance.technews.tw',
       'hk.on.cc', 'house.ettoday.net', 'm.ctee.com.tw', 'm.ltn.com.tw',
       'money.udn.com', 'mops.twse.com.tw', 'news.cnyes.com',
       'news.ebc.net.tw', 'news.ltn.com.tw', 'news.mingpao.com',
       'news.tvbs.com.tw', 'ol.mingpao.com', 'sina.com.hk', 'technews.tw',
       'tw.news.yahoo.com', 'udn.com', 'www.bnext.com.tw',
       'www.businesstoday.com.tw', 'www.chinatimes.com', 'www.cna.com.tw',
       'www.coolloud.org.tw', 'www.cw.com.tw', 'www.ettoday.net',
       'www.fsc.gov.tw', 'www.hbrtaiwan.com', 'www.hk01.com',
       'www.managertoday.com.tw', 'www.mirrormedia.mg',
       'www.nextmag.com.tw', 'www.nownews.com', 'www.setn.com',
       'www.storm.mg', 'www.wealth.com.tw'], dtype=object)

In [5]:
# 單獨測試
index = 0
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
context = getContext(category, url) 
print("url :", url, "\ncategory :", category, "\ncontext :", context)

url : https://news.cnyes.com/news/id/4352432 
category : news.cnyes.com 
context : (0, 0, 0)


In [6]:
soup = getSoup(url)